In [1]:
#!pip install dash

In [40]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("C:\\Users\\Admin\\Documents\\AA_VNU\\Edx_Course\\IBM Data Science\\05_Capstone Project\\spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [39]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    ],
                                    value='ALL',
                                    placeholder="Select a launch site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                           2000: '2000',
                                           4000: '4000',
                                           6000: '6000',
                                           8000: '8000',
                                           10000: '10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        data=filtered_df.groupby('Launch Site')['Launch Site','class'].sum().reset_index()
        fig = px.pie(data, values='class', 
            names='Launch Site', 
            title='Total Success Launches by site')
        return fig
    else:
        data=filtered_df[filtered_df['Launch Site']==entered_site][['Launch Site','class']].groupby('class').count().reset_index()
        fig = px.pie(data, values='Launch Site', 
        names='class', 
        title='Total Success Launches for site')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(entered_site,payload_value):
    filtered_df = spacex_df
    if entered_site == 'ALL':
         data=filtered_df[(filtered_df['Payload Mass (kg)']>payload_value[0])&(filtered_df['Payload Mass (kg)']<payload_value[1])][['Payload Mass (kg)','class','Booster Version Category']]
         fig = px.scatter(data,
                         x=data['Payload Mass (kg)'], 
                         y=data['class'], 
                         color=data['Booster Version Category'],
                         title='Correlation between Pyload Mass and Success for all sites')
         return fig
    else:
        data=filtered_df[(filtered_df['Launch Site']==entered_site)&(filtered_df['Payload Mass (kg)']>payload_value[0])&(filtered_df['Payload Mass (kg)']<payload_value[1])][['Payload Mass (kg)','class','Booster Version Category']]
        fig = px.scatter(data, 
                         x=data['Payload Mass (kg)'], 
                         y=data['class'], 
                         color=data['Booster Version Category'],
                         title='Correlation between Pyload Mass and Success for site'+entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


C:\Users\Admin\AppData\Local\Temp\ipykernel_7516\184516547.py:54: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [20]:
filtered_df = spacex_df
data=filtered_df[(filtered_df['Launch Site']=='CCAFS LC-40')&(filtered_df['Payload Mass (kg)']>2000)][['Payload Mass (kg)','class']]
data

,Payload Mass (kg),class
5,3170.0,0
6,3325.0,0
7,2296.0,0
9,4535.0,0
10,4428.0,0
11,2216.0,0
12,2395.0,0
14,4159.0,0
16,4707.0,0
18,2034.0,1


In [32]:
filtered_df = spacex_df
data=filtered_df[['Payload Mass (kg)','class','Booster Version Category']]
data

,Payload Mass (kg),class,Booster Version Category
0,0.00,0,v1.0
1,0.00,0,v1.0
2,525.00,0,v1.0
3,500.00,0,v1.0
4,677.00,0,v1.0
5,3170.00,0,v1.1
6,3325.00,0,v1.1
7,2296.00,0,v1.1
8,1316.00,0,v1.1
9,4535.00,0,v1.1
